In [2]:
import tweepy
from supabase import create_client, Client
import os
import time
import csv

# Konfigurasi API Twitter
api_key = "PWGm3pML0YeYO8mvJqv13qFsS"  # Ganti dengan os.getenv jika menggunakan variabel lingkungan
api_key_secret = "V58s2t10NtLp8hcbZf6Rm7V5TkOx7ciQPIyb6o484Q2agphjDu"  # Ganti dengan os.getenv jika menggunakan variabel lingkungan
access_token = "1881296539274100737-voP3HtU05deBPcO6rYKOmtU0YNyE7q"  # Ganti dengan os.getenv jika menggunakan variabel lingkungan
access_token_secret = "BGKaMKg6MSVu8Ftg8XTM1Dk7d8YkdLvjhRswknJMy3RSD"  # Ganti dengan os.getenv jika menggunakan variabel lingkungan
bearer_token = "AAAAAAAAAAAAAAAAAAAAAPtgyQEAAAAAlsANjgUZHExz7VChjLwlK3eKF5k%3DRuklKXJCMtAmSQ0fWixiPmneOdcttFWdfRyuvUZ8ErXqRCsyHj"


# Konfigurasi Supabase
supabase_url = "https://dehyfkosyudcfacavvxb.supabase.co"  # Ganti dengan os.getenv jika menggunakan variabel lingkungan
supabase_key = "eeyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJzdXBhYmFzZSIsInJlZiI6ImRlaHlma29zeXVkY2ZhY2F2dnhiIiwicm9sZSI6ImFub24iLCJpYXQiOjE3MzczNzA3MzEsImV4cCI6MjA1Mjk0NjczMX0.Mu3lQHnGpt1Aex-QuzUE2rEVvJ4T8iAq0j1utuBc054"  # Ganti dengan os.getenv jika menggunakan variabel lingkungan
supabase: Client = create_client(supabase_url, supabase_key)

client = tweepy.Client(bearer_token=bearer_token)


# Fungsi untuk scraping tweet
# Fungsi untuk scraping tweet
def scrape_tweets_v2(keyword, max_results):
  """Mengambil tweet menggunakan Twitter API v2.

  Args:
      keyword: Rektor Indonesia
      max_results: 97

  Returns:
      list: Daftar berisi data tweet yang diambil.
  """

  tweets_data = []
  max_per_request = 97  # Maksimal tweet per permintaan API
  pages = max_results // max_per_request  # Berapa kali kita perlu meminta
  next_token = None

  for _ in range(pages):
    try:
      response = client.search_recent_tweets(
          query=keyword,
          max_results=max_per_request,
          tweet_fields=["created_at"],
          next_token=next_token,
      )

      if response.data:
        for tweet in response.data:
          tweets_data.append({
              "tweet": tweet.text,
              "created_at": tweet.created_at,
          })

      # Pagination: Ambil token untuk batch berikutnya
      next_token = response.meta.get("next_token") if response.meta else None
      if not next_token:
        break  # Tidak ada data lagi untuk diambil

      # Jeda sebelum permintaan berikutnya (hindari rate limit)
      time.sleep(1)

    except tweepy.TooManyRequests as e:
      print("Rate limit exceeded")
      time.sleep(3)  # Tidur selama 3 menit sebelum melanjutkan permintaan
    except Exception as e:
      print("Error while scraping tweets:", e)
      break

  return tweets_data

# Fungsi untuk menyimpan data ke Supabase
def save_to_supabase(data):
  """Menyimpan data tweet ke database Supabase.

  Args:
      data: List berisi data tweet yang ingin disimpan.
  """

  for item in data:
    formatted_item = {
      "tweet": item["tweet"],
      "created_at": item["created_at"].isoformat(),
    }
    try:
      response = supabase.table("tweets").insert(formatted_item).execute()
      if response.get("status_code") == 201:
        print("Inserted:", formatted_item)
      else:
        print("Failed to insert:", response)
    except Exception as e:
      print("Error while inserting:", e)

# Fungsi utama
if __name__ == "__main__":
  keyword = "Rektor Indonesia"
  max_results = 1000

  print("Scraping tweets...")
  tweets = scrape_tweets_v2(keyword, max_results)
  print(f"Total tweets scraped: {len(tweets)}")

  print("Saving tweets to Supabase...")
  save_to_supabase(tweets)
  print("Done!")

Scraping tweets...
Rate limit exceeded
Rate limit exceeded
Rate limit exceeded
Rate limit exceeded
Total tweets scraped: 79
Saving tweets to Supabase...
Error while inserting: {'message': 'Invalid API key', 'hint': 'Double check your Supabase `anon` or `service_role` API key.'}
Error while inserting: {'message': 'Invalid API key', 'hint': 'Double check your Supabase `anon` or `service_role` API key.'}
Error while inserting: {'message': 'Invalid API key', 'hint': 'Double check your Supabase `anon` or `service_role` API key.'}
Error while inserting: {'message': 'Invalid API key', 'hint': 'Double check your Supabase `anon` or `service_role` API key.'}
Error while inserting: {'message': 'Invalid API key', 'hint': 'Double check your Supabase `anon` or `service_role` API key.'}
Error while inserting: {'message': 'Invalid API key', 'hint': 'Double check your Supabase `anon` or `service_role` API key.'}
Error while inserting: {'message': 'Invalid API key', 'hint': 'Double check your Supabase `